In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to C:\Users\Roshan
[nltk_data]     Salunke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
data = pd.read_csv(r"C:\Users\Roshan Salunke\Downloads\Data Science Course\Deep Learning\nlp\colab\train (1).csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
x = data.drop('label',axis=1)
y = data['label']

In [13]:
sentences = x.copy()
sentences.reset_index(inplace=True)
ps = PorterStemmer()
corpus = []

for i in range(0, len(sentences)):
    text = re.sub('[^a-zA-Z]',' ', sentences['title'][i])
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    corpus.append(text)
    
corpus[:10]

#sentences = x.copy()
#
#sentences.reset_index(inplace=True)
#
#ps = PorterStemmer()
#
#corpus = []
#
#for i in range(0, len(sentences)):
#    review = re.sub('[^a-zA-Z]',' ', sentences['title'][i])
#    review = review.lower()
#    review = review.split()
#    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#    review = ' '.join(review)
#    corpus.append(review)
#    
#corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [14]:
vocab_size = 5000
onehot = [one_hot(words, vocab_size) for words in corpus]
onehot[:10]

[[4258, 3852, 4067, 4544, 2705, 3644, 3199, 3771, 2821, 765],
 [3681, 3424, 4697, 3002, 4337, 2780, 856],
 [3703, 2291, 2007, 4478],
 [1907, 1806, 2647, 2668, 119, 526],
 [2300, 4337, 1817, 4510, 2222, 4756, 4337, 678, 4065, 2554],
 [4675,
  4830,
  479,
  1461,
  4026,
  2342,
  1186,
  2882,
  2069,
  3026,
  3044,
  4151,
  2644,
  2292,
  856],
 [2595, 2515, 4596, 1470, 566, 2214, 3033, 3343, 2497, 1277, 2334],
 [9, 1025, 2421, 98, 3156, 47, 2342, 1427, 2497, 1277, 2334],
 [1680, 2531, 3106, 2688, 4727, 2310, 4654, 3097, 2342, 23],
 [899, 1469, 2630, 567, 4175, 2547, 3523, 1011]]

In [15]:
length = 30
embedding = pad_sequences(onehot, maxlen=length, padding='pre')
embedding[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 4258, 3852,
        4067, 4544, 2705, 3644, 3199, 3771, 2821,  765],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 3681, 3424, 4697, 3002, 4337, 2780,  856],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 3703, 2291, 2007, 4478],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1907, 1806, 2647, 2668,  119,  526],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 2300, 4337,
        1817, 4510, 2222, 4756, 

In [17]:
model = Sequential()

model.add(Embedding(5000, 40, input_length=30))

model.add(Bidirectional(LSTM(100)))

model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [20]:
x = np.array(embedding)
y = np.array(y)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [22]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/20
192/192 [==============================] - 14s 53ms/step - loss: 0.3309 - accuracy: 0.8438 - val_loss: 0.2013 - val_accuracy: 0.9125
Epoch 2/20
192/192 [==============================] - 10s 51ms/step - loss: 0.1438 - accuracy: 0.9428 - val_loss: 0.1894 - val_accuracy: 0.9210
Epoch 3/20
192/192 [==============================] - 9s 48ms/step - loss: 0.1007 - accuracy: 0.9619 - val_loss: 0.2087 - val_accuracy: 0.9181
Epoch 4/20
192/192 [==============================] - 9s 48ms/step - loss: 0.0778 - accuracy: 0.9717 - val_loss: 0.2671 - val_accuracy: 0.9110
Epoch 5/20
192/192 [==============================] - 9s 47ms/step - loss: 0.0592 - accuracy: 0.9797 - val_loss: 0.2800 - val_accuracy: 0.9117
Epoch 6/20
192/192 [==============================] - 9s 48ms/step - loss: 0.0446 - accuracy: 0.9856 - val_loss: 0.3007 - val_accuracy: 0.9112
Epoch 7/20
192/192 [==============================] - 9s 48ms/step - loss: 0.0311 - accuracy: 0.9910 - val_loss: 0.3945 - val_accuracy: 0.91

In [23]:
model.evaluate(x_test, y_test)

189/189 [==============================] - 2s 10ms/step - loss: 0.7007 - accuracy: 0.9032


[0.700720489025116, 0.903231143951416]

In [24]:
pred = model.predict(x_test)

189/189 [==============================] - 3s 10ms/step


In [27]:
data = pd.read_csv(r"C:\Users\Roshan Salunke\Downloads\Data Science Course\Deep Learning\nlp\colab\train (1).csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [29]:
data.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [30]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [31]:
data.dropna(inplace=True)

In [32]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [33]:
x = data.drop('label', axis=1)
y = data['label']

In [34]:
sentence = x.copy()
sentence.reset_index(inplace=True)
ps = PorterStemmer()
corpus = []
for i in range(0, len(sentence)):
    text = re.sub('[^a-zA-Z]',' ', sentence['title'][i])
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    corpus.append(text)
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [37]:
vocab_size = 5000
onehot = [one_hot(word, vocab_size) for word in corpus]
len(onehot)

18285

In [38]:
maxlen = 30
padd = pad_sequences(onehot, maxlen=maxlen,padding='pre')
padd[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 4258, 3852,
        4067, 4544, 2705, 3644, 3199, 3771, 2821,  765],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 3681, 3424, 4697, 3002, 4337, 2780,  856],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 3703, 2291, 2007, 4478],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1907, 1806, 2647, 2668,  119,  526],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 2300, 4337,
        1817, 4510, 2222, 4756, 

In [39]:
model = Sequential()
model.add(Embedding(5000, 40, input_length=30))

model.add(Bidirectional(LSTM(100)))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [41]:
x = np.array(padd)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [42]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size= 64)

Epoch 1/5
192/192 [==============================] - 14s 53ms/step - loss: 0.3335 - accuracy: 0.8370 - val_loss: 0.1952 - val_accuracy: 0.9150
Epoch 2/5
192/192 [==============================] - 10s 51ms/step - loss: 0.1462 - accuracy: 0.9421 - val_loss: 0.1967 - val_accuracy: 0.9175
Epoch 3/5
192/192 [==============================] - 10s 53ms/step - loss: 0.0938 - accuracy: 0.9657 - val_loss: 0.2019 - val_accuracy: 0.9239
Epoch 4/5
192/192 [==============================] - 10s 51ms/step - loss: 0.0602 - accuracy: 0.9802 - val_loss: 0.2458 - val_accuracy: 0.9145
Epoch 5/5
192/192 [==============================] - 10s 52ms/step - loss: 0.0410 - accuracy: 0.9867 - val_loss: 0.2708 - val_accuracy: 0.9186


In [43]:
model.evaluate(x_test, y_test)

189/189 [==============================] - 2s 10ms/step - loss: 0.2708 - accuracy: 0.9186


[0.2708333432674408, 0.9186412692070007]